## Settings

In [1]:
fieldsFloat = ["Spend", "Revenue", "Budget", "ROAS", "CPI", "CPP", "CohortNum", "DayNum", "Reach", "Impressions", "Clicks", "Installs", "Purchases", "Payers", "Opt_goal_conversion_window", "Age Min", "Age Max"]

game = "Game_11-55"#"Game_3c-e1"
#games = [Game_f0-90, Game_5с-16, Game_11-55, Game_0d-82,
#         Game_14-2e, Game_38-2a, Game_a4-99, Game_a2-fc,
#         Game_bd-be, Game_68-1e, Game_b5-f6, Game_3c-e1,
#         Game_50-58, Game_d9-df, Game_da-e4, Game_a7-b9
# mid: [1, 11, 13]
# large: [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15]

featDays = [2]
targDays = [2]
z_dims = [4]
train_batch_size = 32
N = 32


## Loading functions

In [2]:
import sys
sys.path.insert(0, '../common')
import DataGetter
import pandas as pd
import numpy as np

datagetter = DataGetter.DataGetter()
datagetter.setDataPath("../data2")
games = datagetter.getGamesList()

def dataLoadDays(featDay, targDay):
    # features day and target day
    loadgames = {}
    for game in games:
        (i_revenue_all, i_revenue_not_null, df) = datagetter.getGamesData([game], [], [])
        validInds = pd.Series([False] * len(df))
        for adset in df["Adset"].unique():
            for cohort in df[df["Adset"] == adset]["CohortNum"].unique():
                ind = (df["Adset"] == adset) & (df["CohortNum"] == cohort)
                dft = df[ind]
                if len(dft[dft["DayNum"]==targDay]) > 0:
                    validInds = (validInds) | (ind)
        loadgames[game] = df[validInds]
        
    return loadgames


def dataLoadGameDays(game, featDay, targDay):
    # features day and target day
    (i_revenue_all, i_revenue_not_null, df) = datagetter.getGamesData([game], [], [])
    #validInds = pd.Series([False] * len(df))
    #for adset in df["Adset"].unique():
    #    for cohort in df[df["Adset"] == adset]["CohortNum"].unique():
    #        ind = (df["Adset"] == adset) & (df["CohortNum"] == cohort)
    #        dft = df[ind]
    #        if len(dft[dft["DayNum"]==targDay]) > 0:
    #            validInds = (validInds) | (ind)
    #    
    #return df[validInds]
    dfAcT = df[df["DayNum"]==targDays[0]][["Adset", "CohortNum"]].drop_duplicates()
    dfAcA = df[["Adset", "CohortNum"]]
    dfAcA["common"] = dfAcA["Adset"] + dfAcA["CohortNum"].astype(str)
    dfAcT["common"] = dfAcT["Adset"] + dfAcT["CohortNum"].astype(str)
    dfAcAV = dfAcA["common"].values
    dfAcTV = dfAcT["common"].values
    #print(len(dfAcAV), len(dfAcTV))
    ind = [x in dfAcTV for x in dfAcAV]
    #print(len(ind), sum(ind))
    return df[ind]

In [3]:
(i_revenue_all, i_revenue_not_null, df) = datagetter.getGamesData([game], [], [])

Number of all cohorts: 27561 
  number of cohorts with revenues = zero: 22822 
  number of cohorts with revenues != zero: 4739


In [4]:
#df = dataLoadGameDays(game, featDays[0], targDays[0])
#df = df[df["DayNum"]==featDays[0]]

In [ ]:
import torch
import pickle
import numpy as np
import pandas as pd
import math

import sys
sys.path.insert(0, '../common')
import DataGetter

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


def getDfWithFeatures2(game, targDay, advancedFeatures=False):
    datagetter = DataGetter.DataGetter()
    datagetter.setDataPath("../data2")
    (i_revenue_all, i_revenue_not_null, df) = datagetter.getGamesData([game], [], [])

    dfAcT = df[df["DayNum"]==targDays[0]][["Adset", "CohortNum"]].drop_duplicates()
    dfAcA = df[["Adset", "CohortNum"]]
    dfAcA["common"] = dfAcA["Adset"] + dfAcA["CohortNum"].astype(str)
    dfAcT["common"] = dfAcT["Adset"] + dfAcT["CohortNum"].astype(str)
    dfAcAV = dfAcA["common"].values
    dfAcTV = dfAcT["common"].values
    #print(len(dfAcAV), len(dfAcTV))
    ind = [x in dfAcTV for x in dfAcAV]
    #print(len(ind), sum(ind))

    cols = [c for c in df.columns if c not in ["Adset", "DateCohort", "DateDay", "Date", "CampId", "AccId"]]
    df = df[ind][cols]

    df = df.replace([math.inf, -math.inf], math.nan)
    df = df.fillna(0)

    return df


def load_full_data(game, featDay=2, targDay=13, targFeature="Payers", train_batch_size=32):
    print('loading data!')
    df = getDfWithFeatures2(game, targDay)
    
    # features types processing
    columns = df.columns
    columnsToDel = []
    for c in columns:
        if len(np.unique(df[c])) == 1:
            columnsToDel.append(c)
    df = df[[c for c in columns if c not in columnsToDel]]
    columns = df.columns
    #
    columnsToDel = []
    for c in columns:
        #try:
        #    np.unique(df[c])
        #except:
        #    print(c)
        #    print(df[c])
        if type(df.iloc[0][c]) == str:
            df[c] = df[c].astype(str)
            if len(np.unique(df[c])) == 2:
                le = LabelEncoder()
                le.fit(df[c])
                df[c] = le.transform(df[c])
            else:
                enc = OneHotEncoder(sparse=False)
                new_ohe_features = enc.fit_transform(df[c].values.reshape(-1, 1))
                tmp = pd.DataFrame(new_ohe_features, columns=[c+"_"+str(i) for i in range(new_ohe_features.shape[1])])
                df = pd.concat([df, tmp], axis=1)
                columnsToDel.append(c)
    columns = df.columns
    df = df[[c for c in columns if c not in columnsToDel]]
    #print(df)
    columns = df.columns
    #print(df)
    print(len(df.columns))
    
    # fill load list
    feat_keys = [c for c in df.columns if c != "DayNum" and c != targFeature]
    
    for f in df.columns:
        if type(df.iloc[0][f]) == str or type(df.iloc[0][f]) == object:
            df[f] = df[f].fillna("")
            df[f] = np.where(pd.isnull(df[f].values), "", df[f].values)
        else:
            df[f] = df[f].fillna(0)
            df[f] = np.where(pd.isnull(df[f].values), 0, df[f].values)

    #df['norm'] = np.where(df['Impressions'] <= 0, 1, df['Impressions'])
    #print(df['norm'])
    #for c in df.columns:
    #    if c in fieldsFloat and c != "norm":
    #        df[c] = df[c] / df["norm"]
    #df = df[[c for c in columns if c != "norm" and c != "Impressions"]]
    #feat_keys = [f for f in feat_keys if f != "Impressions"]
    
    nparr = df[feat_keys + [targFeature]].values
    maxs = np.max(nparr, axis = 0)
    load_list = []
    
    #print(df.columns, maxs, df)
    
    #falseCols = []
    #for c in df.columns:
    #    if np.sum(pd.isnull(df[c])):
    #        print(c)
    #        falseCols.append(c)
    #print(maxs)
    #print(df[falseCols])

    
    for i in range(len(df)):
        if df.iloc[i]['DayNum'] == featDay:
            lst = df.iloc[i][feat_keys].tolist()
            lst.append(df.iloc[i + targDay - featDay][targFeature])
            i = i + targDay - featDay
            # print(lst)
            load_list.append(lst)

    # normalize list by maxs
    i = 0
    for lst in load_list:
        lst_ = np.array(lst)
        lst_ = lst_ / maxs
        load_list[i] = lst_.tolist()
        #print(lst_, maxs, load_list[i])
        i = i + 1

    # fill array with train_batch_size
    if train_batch_size>0:
        data = np.zeros(shape=(int(len(load_list) / train_batch_size), train_batch_size, len(feat_keys) + 1), dtype=float)
        dt = np.zeros(shape=(train_batch_size, len(feat_keys) + 1), dtype=float)
        upindex = 0
        index = 0
        for lst in load_list:
            dt[index, :] = lst
            index = index + 1
            if index == train_batch_size:
                index = 0
                #print(dt)
                data[upindex, :, :] = dt
                # if upindex==0:
                #    print(dt)
                dt = np.zeros(shape=(train_batch_size, len(feat_keys) + 1), dtype=float)
                upindex = upindex + 1

        data = torch.from_numpy(data)  # learn_list)
        # X = Variable(torch.from_numpy(learn_list).float(), requires_grad = True)
    else:
        data = torch.from_numpy(np.array(load_list))

    return (data, maxs, feat_keys+[targFeature])



In [ ]:
import pickle
#import sys
#sys.path.insert(0, '../common')
#from DataLoaderForEncoder import load_data

(data, maxs, cols) = load_full_data(game, featDays[0], targDays[0], "Payers", 32)
#cols = [c for c in cols if c!= "Impressions"]
#with open('./pickles/maxs_' + games[0] + '_%d_%d__.pickle' % (featDays[0], targDays[0]), 'wb') as f:
#                    pickle.dump(maxs, f)
with open('maxs_' + game + '_%d_%d__.pickle' % (featDays[0], targDays[0]), 'wb') as f:
                    pickle.dump(maxs, f)
#data

loading data!
Number of all cohorts: 27561 
  number of cohorts with revenues = zero: 22822 
  number of cohorts with revenues != zero: 4739


/home/sdeev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


211


In [ ]:
maxs

array([1.61000e+02, 1.00000e+00, 9.21440e+02, 9.95810e+02, 1.11485e+03,
       1.39191e+05, 1.57936e+05, 2.33200e+03, 7.65000e+02, 9.50000e+01,
       6.70340e+02, 4.23100e+01, 2.73030e+02, 1.00000e+00, 7.00000e+00,
       2.50000e+01, 6.50000e+01, 1.00000e+00, 1.00000e+00, 1.00000e+00,
       1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00,
       1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00,
       1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00,
       1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00,
       1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00,
       1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00,
       1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00,
       1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00,
       1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00,
       1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.000

In [ ]:
import torch
import pickle
import numpy as np
from torch.autograd import Variable

import sys
sys.path.insert(0, '../encoder1')
from VaeEncoder import VaeEncoder, VAE2

#print(data.shape[1])
vae = VaeEncoder(X_dim=data.shape[2], N=32, z_dim=z_dims[0])
(model, losses) = vae.generate_model(data)

Epoch-0; loss: 8.987e+05
Epoch-10; loss: 3.795e+05
Epoch-20; loss: 3.608e+05
Epoch-30; loss: 3.529e+05
Epoch-40; loss: 3.465e+05
Epoch-50; loss: 3.429e+05
Epoch-60; loss: 3.402e+05
Epoch-70; loss: 3.376e+05
Epoch-80; loss: 3.353e+05
Epoch-90; loss: 3.33e+05
Epoch-100; loss: 3.315e+05
Epoch-110; loss: 3.303e+05
Epoch-120; loss: 3.29e+05
Epoch-130; loss: 3.279e+05
Epoch-140; loss: 3.265e+05
Epoch-150; loss: 3.257e+05
Epoch-160; loss: 3.252e+05
Epoch-170; loss: 3.242e+05
Epoch-180; loss: 3.235e+05
Epoch-190; loss: 3.236e+05
Epoch-200; loss: 3.219e+05
Epoch-210; loss: 3.225e+05
Epoch-220; loss: 3.217e+05
Epoch-230; loss: 3.22e+05
Epoch-240; loss: 3.205e+05
Epoch-250; loss: 3.202e+05
Epoch-260; loss: 3.196e+05
Epoch-270; loss: 3.187e+05
Epoch-280; loss: 3.188e+05
Epoch-290; loss: 3.191e+05
Epoch-300; loss: 3.19e+05
Epoch-310; loss: 3.177e+05
Epoch-320; loss: 3.188e+05
Epoch-330; loss: 3.164e+05
Epoch-340; loss: 3.171e+05
Epoch-350; loss: 3.168e+05
Epoch-360; loss: 3.181e+05
Epoch-370; loss:

## Generate

In [ ]:
import sys
sys.path.insert(0, '../common')
import DataGetter
import pandas as pd

datagetter = DataGetter.DataGetter()
datagetter.setDataPath("../data")
games = datagetter.getGamesList()

#columns = ["Spend", "Reach", "Impressions", "Clicks", "Installs", "Payers"]

maxDensityCells = 1600
samplesCount = 25000

In [ ]:
import torch
from DataLoaderForEncoder import load_data

def loadGameDataForDensity(game, featDay, targDay):
    (data, norms, cols) = load_full_data(game, featDay, targDay, train_batch_size=0)

    print(type(data), data.shape)
    data = data.float()
    data = torch.autograd.Variable(data)
    if torch.cuda.is_available():
        data = data.cuda()

    return (data, norms, cols)

In [ ]:
import numpy as np
import pandas as pd
import math
import torch
import sys
sys.path.insert(0, '../encoder1')
#import AaeEncoder

#print(__doc__)
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA

def getHiddenGameLayerForDensity(game, featDay=2, targDay=13, hiddenNeurons=2):
    (data, norms, cols) = loadGameDataForDensity(game, featDay, targDay)
    #"VAE__;seed_10;z_4;N_32;batch_32;opt_Adam;dropout=0.100000.pickle"
    #qName = 'VAE_' + game + ('_%d_%d_' % (featDay, targDay)) + \
    qName = 'VAE_' + \
            ('_;seed_10;z_%d;N_%d;batch_%d;opt_Adam;dropout=0.100000.pickle' % (z_dims[0], N, train_batch_size))
    #qName = "VAE__;seed_10;z_2;N_32;batch_32;opt_Adam;dropout=0.100000.pickle"
    model = torch.load(qName)
    #result = model(data) # ret of forward
    result = model.encode(data) # mu, logvar
    #if torch.cuda.is_available():
    #    result = result.cpu()

    res = []
    if torch.cuda.is_available():
        res.append(result[0].cpu().data.numpy())
        res.append(result[1].cpu().data.numpy())
    else:
        res.append(result[0].data.numpy())
        res.append(result[1].data.numpy())
    
    return res

In [ ]:
result = getHiddenGameLayerForDensity(game, featDays[0], targDays[0])
#result

loading data!
Number of all cohorts: 27561 
  number of cohorts with revenues = zero: 22822 
  number of cohorts with revenues != zero: 4739


/home/sdeev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


211
<class 'torch.Tensor'> torch.Size([27487, 210])


### MDS of hidden layer

In [ ]:
import matplotlib.pyplot as plt

X=result[0].tolist()
mds=manifold.MDS(n_components=2)
X_r=mds.fit_transform(X)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 8))
ax.scatter([x[0] for x in X_r], [x[1] for x in X_r])
plt.show()

print(len(X_r[0]))

### Distances between samples in hidden layer

In [ ]:
import matplotlib.pyplot as plt

rs = []
#X = result[0].tolist()
n = len(X)
m = len(X[0])
for i in range(n - 1):
    for j in range(i + 1, n):
        x = X[i]
        y = X[j]
        rs.append(sum([(x[k]-y[k])*(x[k]-y[k]) for k in range(m)]))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
ax.hist(rs)
plt.show()

### Weights of columns for hidden layer: dR_{hidden} / dCol

In [ ]:
import numpy as np

(data, norms, cols) = loadGameDataForDensity(game, featDays[0], targDays[0])
datan  = data.data.numpy()
#normsn = norms

deltasData = np.zeros([datan.shape[1] + 1, datan.shape[1]]) # len(cols)+1, len(cols)
means = np.mean(datan, axis=0)
maxs  = np.max(datan, axis=0)
mins  = np.min(datan, axis=0)
dels  = maxs - mins
deltasData[0] = means
for i in range(datan.shape[1]):
    delta = [0] * datan.shape[1]
    delta[i] = dels[i] / 100 #0.01
    deltasData[1 + i] = means + np.array(delta)
deltasData = torch.from_numpy(deltasData)
deltasData = deltasData.float()
deltasData = torch.autograd.Variable(deltasData)
if torch.cuda.is_available():
    deltasData = deltasData.cuda()

qName = 'VAE_' + \
        ('_;seed_10;z_%d;N_%d;batch_%d;opt_Adam;dropout=0.100000.pickle' % (z_dims[0], N, train_batch_size))

model = torch.load(qName)

result = model.encode(deltasData) # mu, logvar

In [ ]:
if torch.cuda.is_available():
    Xnp = result[0].cpu().data.numpy()
else:
    Xnp = result[0].data.numpy()

weights = []
for i in range(len(Xnp) - 1):
    weights.append( sum((Xnp[i+1]-Xnp[0]) * (Xnp[i+1]-Xnp[0]) ) )
    
weightsSer = pd.Series(weights, cols)
weightsSer

### Correlations of columns with Payers of hidden layer
We have vector in hidden layer with dPayers and vectors in hidden layer with dColumns
Correlation is scalar productions of these vectors

In [ ]:
rHidPayers = Xnp[-1] - Xnp[0]
rHidCols = []
for i in range(len(Xnp) - 2):
    rHidCols.append(Xnp[i+1] - Xnp[0])

corrs = [ sum(rHidPayers * rHidCol) * 10000 for rHidCol in rHidCols]
corrsSer = pd.Series(corrs, cols[:-1])
corrsSer

### Distribution of Payers with generation from center point in hidden layer

In [ ]:
import math
import numpy as np
import random
import torch

mu = X[0]
logvar = result[1][0] # logvar.exp() = sigma2
if torch.cuda.is_available():
    logvar = logvar.cpu().data.numpy()
else:
    logvar = logvar.data.numpy()
sigma = np.sqrt(np.exp(logvar))

def f(x):
    hidPoint = np.random.normal(mu, sigma)
    hidPoint = Variable(torch.from_numpy(np.array(hidPoint)).float())
    gen = model.decode(hidPoint)
    if torch.cuda.is_available():
        return gen.cpu().data.numpy() * norms
    else:
        return gen.data.numpy() * norms
     
g = np.vectorize(f, otypes=[np.ndarray])
a = np.arange(10000)
generated = g(a)
#print(generated)

In [ ]:
import matplotlib.pyplot as plt

plt.hist([g[-1] for g in generated])
plt.show()

### Clusters in hidden layer research

#### DBSCAN

In [ ]:
from sklearn import cluster
import matplotlib.pyplot as plt

def dbscan(points):
    dbscan = cluster.DBSCAN(eps=0.3, min_samples=3)
    dbscan.fit(points)
    pointLabels = dbscan.labels_
    core = dbscan.core_sample_indices_
    #print( repr(core) )
    size = [5 if i not in core else 40 for i in range(len(X))]
    #print( repr(size) )
    return pointLabels

pointLabels = dbscan(X)

mds=manifold.MDS(n_components=2)
X_r=mds.fit_transform(X)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 8))
ax.scatter([x[0] for x in X_r], [x[1] for x in X_r], 
            c=['bgrcmyk'[pointLabels[i] % 7] for i in range(len(X_r))],
            alpha=0.8, marker='o')
plt.show()


#### AffinityPropagation clustering (for example)

In [ ]:
from sklearn.cluster import AffinityPropagation
import matplotlib.pyplot as plt
import numpy as np

def affclust(points):
    clustering = AffinityPropagation(preference=-45).fit(points)
    return clustering.labels_

pointLabels = affclust(X)

mds=manifold.MDS(n_components=2)
X_r=mds.fit_transform(X)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 8))
ax.scatter([x[0] for x in X_r], [x[1] for x in X_r], 
            c=['bgrcmyk'[pointLabels[i] % 7] for i in range(len(X_r))],
            alpha=0.8, marker='o')
plt.show()

#### Mean features values for clusters

In [ ]:
pointLabels = dbscan(X)
#print(np.unique(pointLabels))

(data, norms, cols) = load_full_data(game, featDays[0], targDays[0], train_batch_size=0)
data = data.data.numpy()
data = data * norms


In [ ]:
df = pd.DataFrame(index=(["N points"]+cols), columns=[str(cl+1) for cl in range(max(pointLabels))])
for cluster in range(max(pointLabels) + 1):
    ind = (np.array(pointLabels) == cluster)
    Npoints = sum(ind)
    dt = data[ind]
    #print(Npoints, ind, len(dt))
    #print(np.mean(dt, axis=0))
    df.loc["N points"]["%d" % (cluster+1)] = Npoints
    means = np.mean(dt, axis=0)
    for i in range(len(cols)):
        df.loc[cols[i]]["%d" % (cluster+1)] = means[i]
    #print(df)
    #break
df


In [ ]:
bts = b"12;34;1\n234"
bts.index(b"\n")
bts[:bts.index(b"\n")].decode("utf-8").split(";")